# GPT-2 Model

In [1]:
import torch
import torch.nn as nn

In [4]:
class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.esp = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))
    
    def forward(self, x):
        mean = x.mean(dim=-1, keepdims=True)
        var = x.var(dim=-1, keepdims=True, unbiased=False)  #unbiased parameter to replicate GPT2, does devide by N not N-1
        x_norm = (x - mean) / torch.sqrt(var + self.esp)
        return self.scale * x_norm + self.scale 

In [5]:
# activation used in GPT-2 model
class GELU(nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
                                    torch.sqrt(torch.tensor(2.0 / torch.pi)) *
                                    (x + 0.44715 * torch.pw(x, 3))
                                    )
                      )

In [8]:
# simple feedForwaed network
class FeedForward(nn.Module):
    def __init__(self, d_in, d_hidden=None):
        super().__init__()
        if d_hidden is None: d_hidden = 4 * d_in
        self.layers = nn.Sequential(
            nn.Linear(d_in, d_hidden),
            GELU(),
            nn.Linear(d_hidden, d_in)
        )
    
    def forward(self, x):
        return self.layers(x)

In [52]:
# Muti head self attention from pervious session - check MHSAOptimized

class MHSA(nn.Module):
    def __init__(self, d_in, d_out, context_len, n_heads=2, causal=True, dropout=0.1, qkv_bias=False):
        super().__init__()
        assert (d_out % n_heads == 0), 'd_out must be a multiple of n_heads'
        # self.d_in = d_in
        self.d_out = d_out
        self.n_heads = n_heads
        self.causal = causal
        # self.dropout = dropout
        self.head_dim = d_out // n_heads
        
        # k, q, v
        self.query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.value = nn.Linear(d_in, d_out, bias=qkv_bias)
        
        # liner projection, not a nesseary projection, can be skipped
        self.out_proj = nn.Linear(d_out, d_out)
        
        # dropout
        self.dropout = nn.Dropout(dropout)
        
        if self.causal:
            self.register_buffer(
                "mask",
                torch.triu(torch.ones(context_len, context_len),
                        diagonal=1)
            )
            
    def forward(self, x):
        b, n_tokens, d_in = x.shape
        q = self.query(x)
        k = self.key(x)
        v = self.value(x)
        
        # reshaping for n_heads attention, and transpose
        # (b, n_tokens, self.n_heads, self.head_dim) -> (b, self.n_heads, n_tokens, self.head_dim)
        q = q.view(b, n_tokens, self.n_heads, self.head_dim).transpose(1, 2)
        k = k.view(b, n_tokens, self.n_heads, self.head_dim).transpose(1, 2)
        v = v.view(b, n_tokens, self.n_heads, self.head_dim).transpose(1, 2)
        
        att_score = q @ k.transpose(2, 3) # out -> (b, self.n_heads, n_tokens, n_tokens)
        
        # MASKING future attention score, replace with inf that will be changed to zero by softmax
        if self.causal:
            att_score.masked_fill_(self.mask.bool()[:n_tokens, :n_tokens], float('-inf'))
        att_weights = torch.softmax(att_score/k.shape[-1]**0.5, dim=-1)
        
        # dropout
        att_weights = self.dropout(att_weights)
        
        # context vectors
        # -> (b, self.n_heads, n_tokens, self.head_dim) -> (b, n_tokens, self.n_heads, self.head_dim)
        context_vec = (att_score @ v).transpose(1, 2)
        # print(context_vec)   
        
        context_vec = context_vec.contiguous().view(b, n_tokens, self.d_out)
        
        # liner projection
        context_vec = self.out_proj(context_vec)
        
        return context_vec

In [53]:
class TransformerLayer(nn.Module):
    def __init__(self, conf):
        super().__init__()
        self.attn = MHSA(
            d_in = conf.get('emb_dim'),
            d_out = conf.get('emb_dim'),
            context_len = conf.get('context_len'),
            n_heads = conf.get('n_heads'),
            dropout = conf.get('drop_rate'),
            causal = True,
            qkv_bias = conf.get('qkv_bias')
        )
        
        self.ff = FeedForward(conf.get('emb_dim'))
        self.norm1 = LayerNorm(conf.get('emb_dim'))
        self.norm2 = LayerNorm(conf.get('emb_dim'))
        self.dropout = nn.Dropout(conf.get('drop_rate'))
    
    def forward(self, x):
        shortcut = x
        x = self.norm1(x)
        x = self.attn(x)
        x = self.dropout(x)
        x = x + shortcut

        shortcut = x
        x = self.norm2(x)
        x = self.attn(x)
        x = self.dropout(x)
        x = x + shortcut
        
        return x

In [55]:
class GPT2(nn.Module):
    def __init__(self, conf):
        super().__init__()
        self.tok_emb = nn.Embedding(conf.get('vocab_size'), conf.get('emb_dim'))
        self.pos_emb = nn.Embedding(conf.get('context_len'), conf.get('emb_dim'))
        self.drop_emb = nn.Dropout(conf.get('drop_rate'))
        
        self.trf_block = nn.Sequential(
            * [ TransformerLayer(conf) for _ in range(conf.get('n_layers')) ]
        )       # TODO: why * is needed here?
        
        self.final_norm = LayerNorm(conf.get('emb_dim'))
        self.out_head = nn.Linear(conf.get('emb_dim'), conf.get('vocab_size'), bias=False)
        
        
    def forward(self, x):
        batch_size, seq_len = x.shape
        tok_embeds = self.tok_emb(x)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=x.device))
        
        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.trf_block(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits

# Model Configrations


In [56]:
gpt2_conf_124M = {
    'vocab_size': 50257,
    'context_len': 1024,
    'emb_dim': 768,
    'n_heads': 12,
    'n_layers': 12,
    'drop_rate': 0.01,
    'qkv_bias': False,
}

In [57]:
torch.manual_seed(123)
model = GPT2(gpt2_conf_124M)

In [59]:
total_param = sum(p.numel() for p in model.parameters())
print(f"Total Parameters: {total_param}")

Total Parameters: 163009536


In [61]:
# after weight tying
gpt2_params = (
    total_param - sum(p.numel() for p in model.out_head.parameters())
)

print(f"Total Trainable param in GPT2 model: {gpt2_params}")

Total Trainable param in GPT2 model: 124412160


In [51]:
# batch = torch.randint(1, 50000, (2,5))
inputs = [[6109, 3626, 6100, 345],
          [6109, 1110, 6622, 257]]
batch = torch.tensor(inputs)
out = model(batch)
out, out.shape

(tensor([[[nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan]],
 
         [[nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan]]], grad_fn=<UnsafeViewBackward0>),
 torch.Size([2, 4, 50257]))